# Final Assignment AutoGIS 2018
##### Name: Ella Rauth 
##### Student ID: 014964527

## Importing Packages and Data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# reading the grid file
grid = gpd.read_file('data/MetropAccess_YKR_grid/MetropAccess_YKR_grid_EurefFIN.shp')
grid.head(5)

,x,y,YKR_ID,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.00013875 6697750.000128181, 3..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.0001386951 6697750.000128172,..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.0001386406 6697750.000128165,..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000138978 6697500.000128254, ..."


## NoData Values
Notice that there are NoData values present in the data (value -1). In such cases the result cell should always end up having a value -1 when doing travel mode comparisons. In the visualizations, the NoData values should be removed before visualizing the map.

## Functionality 1
AccessViz finds from the data folder all the matrices that user has specified by assigning a list of integer values that should correspond to YKR-IDs found from the attribute table of a Shapefile called MetropAccess_YKR_grid.shp. If the ID-number that the user has specified does not exist in the data folders, the tools should warn about this to the user but still continue running. The tool should also inform the user about the execution process: tell the user what file is currently under process and how many files there are left (e.g. “Processing file travel_times_to_5797076.txt.. Progress: 3/25”).

In [14]:
# saving the YKR_IDs as a list
ykr_ids = list(grid.YKR_ID)

# user defined list of YKR_IDs
input_ids = [5785642, 123, 5793269, 5810402]

# new lists for valid and invalid ids in input_ids
valid_ids = []
invalid_ids = []

# try to match input_ids with ykr_ids and split ids up into valid and invalid
for i in input_ids:
    if i in ykr_ids:
        valid_ids.append(i)
    else:
        invalid_ids.append(i)
        print(i, 'is not a valid YKR-ID!')
        
# inform the user of the numer of valid IDs
print('\nThe following', len(valid_ids), 'valid IDs are going to be processed:\n', valid_ids)

# do something with the travel time matrices of the valid ids:
    # inform user about execution process:
        # which file is currently under process
        # how many files are left
        # example: Processing file travel_times_to_5797076.txt Progress: 3/25 files processed

123 is not a valid YKR-ID!

The following 3 valid IDs are going to be processed:
 [5785642, 5793269, 5810402]


## Functionality 2

AccessViz can create Shapefiles from the chosen Matrix text tables (e.g. travel_times_to_5797076.txt) by joining the Matrix file with MetropAccess_YKR_grid Shapefile where from_id in Matrix file corresponds to YKR_ID in the Shapefile. The tool saves the result in the output-folder that user has defined. You should name the files in a way that it is possible to identify the ID from the name (e.g. 5797076).

In [18]:
# intitializing counter for processed files
done = 0

# processing the travel time matrices for the valid ids
for i in valid_ids:
    # joining the current travel time matrix with the grid file on from_id and YKR_ID
    # join_result = 

    # specifying the output folder
    output_folder = 'output'

    # creating file name
    output_file_name = output_folder + '/travel_times_to_' + str(i) + '.shp'
    
    # saving the file
    # join_result.to_file(output_file_name)
    
    # incrementing counter of processed files
    done += 1
    
    # informing the user about the progress
    print('Processed file travel_times_to_'+str(i)+'.txt', 
          'Progress:', str(done)+'/'+str(len(valid_ids))+' files processed')

Processed file travel_times_to_5785642.txt Progress: 1/3 files processed
Processed file travel_times_to_5793269.txt Progress: 2/3 files processed
Processed file travel_times_to_5810402.txt Progress: 3/3 files processed


## Functionality 3

AccessViz can visualize the travel times of selected YKR_IDs based on the travel mode that the user specifies. It can save those maps into a folder that user specifies. The output maps can be either static or interactive and user can choose which one with a parameter. You can freely design yourself the style of the map, colors, travel time intervals (classes) etc. Try to make the map as informative as possible!

## Functionality 4

AccessViz can also compare travel times or travel distances between two different travel modes (more than two travel modes are not allowed). Thus IF the user has specified two travel modes (passed in as a list) for the AccessViz, the tool will calculate the time/distance difference of those travel modes into a new data column that should be created in the Shapefile. The logic of the calculation is following the order of the items passed on the list where first travel mode is always subtracted by the last one: travelmode1 - travelmode2. The tool should ensure that distances are not compared to travel times and vice versa. If the user chooses to compare travel modes to each other, you should add the travel modes to the filename such as Accessibility_5797076_pt_vs_car.shp. If the user has not specified any travel modes, the tool should only create the Shapefile but not execute any calculations. It should be only possible to compare two travel modes between each other at the time. Accepted travel modes are the same ones that are found in the actual TravelTimeMatrix file (pt_r_tt, car_t, etc.). If the user specifies something else, stop the program, and give advice what are the acceptable values.

## Functionality 5

(option 1). Bundled with AccessViz there is also a separate interactive map that shows the YKR grid values in Helsinki region. The purpose of the map is to help the user to choose the YKR-IDs that s/he is interested to visualize / analyze.

## Functionality 6

(option 2). AccessViz can also visualize the travel mode comparisons that were described in step 4. You can design the style of the map yourself, but try to make it as informative as possible!

## Functionality 7

(option 3). AccessViz can also visualize shortest path routes (walking, cycling, and/or driving) using OpenStreetMap data from Helsinki Region. The impedance value for the routes can be distance (as was shown in Lesson 7) or time (optional for the most advanced students). This functionality can also be a separate program (it is not required to bundle include this with the rest of the AccessViz tool)